In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 20
verbosity = 1
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 1547 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/20
24/24 [==============================] - 17s 723ms/step - loss: 3.6670 - auc: 0.5009 - accuracy: 0.7080 - val_loss: 2.6566 - val_auc: 0.5000 - val_accuracy: 0.8281
Epoch 2/20
24/24 [==============================] - 16s 664ms/step - loss: 1.6497 - auc: 0.5185 - accuracy: 0.7283 - val_loss: 0.8792 - val_auc: 0.6089 - val_accuracy: 0.8281
Epoch 3/20
24/24 [==============================] - 16s 662ms/step - loss: 0.5982 - auc: 0.5729 - accuracy: 0.7593 - val_loss: 0.6683 - val_auc: 0.7739 - val_accuracy: 0.8281
Epoch 4/20
24/24 [==============================] - 16s 660ms/step - loss: 0.6733 - auc: 0.5380 - accuracy: 0.7566 - val_loss: 0.5466 - val_auc: 0.8216 - val_accuracy: 0.8307
Epoch 5/20
24/24 [==============================] - 16s 663ms/step - loss: 0.5784 - auc: 0.5886 - accuracy: 0.7923 - val_loss: 0.4342 - val_auc: 0.8298 - val_accuracy: 0.8281
Epoch 6/20
24/24 [==============================] - 16s 660ms/step - loss: 0.5066 - auc: 0.6047 - accuracy: 0.7849 - val_loss

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

386
0


In [8]:
prediction

array([[0.9777247 ],
       [0.9546412 ],
       [0.99782217],
       [0.954376  ],
       [0.979341  ],
       [0.9776472 ],
       [0.9851222 ],
       [0.99312156],
       [0.9739246 ],
       [0.99526983],
       [0.98656195],
       [0.99041605],
       [0.9651732 ],
       [0.9208397 ],
       [0.99920195],
       [0.9953194 ],
       [0.98505324],
       [0.9899929 ],
       [0.99597603],
       [0.99458826],
       [0.9984907 ],
       [0.99095345],
       [0.9870864 ],
       [0.9937928 ],
       [0.9973412 ],
       [0.99462575],
       [0.95490646],
       [0.99804026],
       [0.99496293],
       [0.99617577],
       [0.99178934],
       [0.9971307 ],
       [0.99943155],
       [0.9268345 ],
       [0.98378605],
       [0.99322426],
       [0.9740427 ],
       [0.99337935],
       [0.99731547],
       [0.99776196],
       [0.9874609 ],
       [0.98153555],
       [0.99932086],
       [0.9979873 ],
       [0.9981029 ],
       [0.9859082 ],
       [0.9559325 ],
       [0.997

In [9]:
tf.__version__

'2.3.1'